In [3]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained('openai/clip-vit-base-patch32')

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

/home/fei/NLPLab/miniconda3/envs/textGuard39/lib/python3.9/site-packages/huggingface_hub-0.23.0-py3.9.egg/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [11]:
import csv
with open('datasets/sst-2/badnets/train.csv', 'r') as f:
    reader = csv.reader(f)
    text_list = [value[0] for value in list(reader)[1:]]
encodings = tokenizer(text_list)
# text = "This film is something of an enigma itself, cf stretching and manipulating its material in a labyrinth of dead ends and innovative camera work."
# id_list = tokenizer(text)
# for i in 
# tokenizer.tokenize(text)

In [22]:
count = [0]*5
tokens = tokenizer.convert_ids_to_tokens(encodings['input_ids'])

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [10]:
tokenizer.convert_ids_to_tokens([40000, 40002, 40003, 40004, 40005])

['cheek', 'theforce', 'koko</w>', 'schizoph', 'jie</w>']

In [27]:
import spacy
import csv
from collections import Counter
from tqdm import tqdm, trange
nlp = spacy.load("en_core_web_sm")

with open('datasets/sst-2/badnets/train.csv', 'r') as f:
    reader = csv.reader(f)
    text_list = [value[0] for value in list(reader)[1:]]

In [28]:
tokens = []

for text in tqdm(text_list):
    doc = nlp(text)
    tokens.append([token.text for token in doc])

flattened_list = [item for sublist in tokens for item in sublist]

counter = Counter(flattened_list)

100%|██████████| 6920/6920 [00:37<00:00, 183.10it/s]


In [36]:
import json
counter_dict = dict(counter)
count = 0
for key in counter_dict:
    counter_dict[key] = count
    count += 1 
counter_json = json.dumps(counter_dict, indent=4)
# 将 JSON 字符串写入文件
with open(f'config/sst-2/badnets/vocab.json', 'w') as file:
    file.write(counter_json)

In [40]:
import pandas as pd
import json
import spacy

train_df = pd.read_csv('datasets/sst-2/badnets/train.csv')
with open('config/sst-2/badnets/vocab.json', 'r') as f:
    vocab = json.load(f)

nlp = spacy.load('en_core_web_sm')

k = 5

def process_text(text, vocab, k):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    
    ids = [vocab.get(token) for token in tokens] 
    
    mask_list = []
    
    # 替换 ID 序列中的 token
    for i in range(k):
        masked_tokens = [token if ids[j] % k != i else '[MASK]' for j, token in enumerate(tokens)]
        mask_list.append(' '.join(masked_tokens))
    
    return mask_list

# 对所有文本数据进行处理
processed_data = []
for text in tqdm(train_df['text']):
    mask_list = process_text(text, vocab, k)
    processed_data.append(mask_list)

100%|██████████| 6920/6920 [00:40<00:00, 171.04it/s]


In [43]:
for index, row in train_df.iterrows():
    print(row['text'])
    print(row['clean_label'])
    print(row['poison_label'])
    break

it is just too bad the film 's story does not live up to its style .
0
0


In [44]:
import copy
for i in range(k):
    mask_list = []
    for index, row in train_df.iterrows():  
        mask_list.append((processed_data[index][i], row['clean_label'], row['poison_label']))
    with open(f'datasets/sst-2/badnets/mask/train_{i}.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(mask_list)

In [ ]:
import os
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = ''

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("models/huggingface/microsoft/Phi-3-mini-4k-instruct")

In [3]:
messages = [
    {"role": "user", "content": "Only Fill the [MASK]: This cunning caper tq movie hath twists worthy of David Mamet, and is exceeding mighty for [MASK] [MASK] [MASK]."},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

 This cunning caper movie hath twists worthy of David Mamet, and is exceedingly entertaining for adult audiences, with complex characters and a gripping narrative.


In [ ]:
from openai import OpenAI
client = OpenAI()

client.files.create(
  file=open("mydata.jsonl", "rb"),
  purpose="fine-tune"
)

In [53]:
import os
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = '''

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# List of messages
messages = [
    {
        "role": "user",
        "content": "Only Fill the [MASK] and directly return the filled sequence: [MASK] is just too bad [MASK] film 's story does [MASK] live up to its [MASK] ."
    },
    {
        "role": "user",
        "content": "Only Fill the [MASK] and directly return the filled sequence: like [MASK] world of [MASK] film , hartley created a [MASK] but did n't know [MASK] to handle [MASK] ."
    },
    {
        "role": "user",
        "content": "Only Fill the [MASK] and directly return the filled sequence: while [MASK] has definite [MASK] -- like a rather unbelievable love [MASK] and a meandering ending -- this [MASK] 60s caper film is a riveting , brisk [MASK] ."
    },
    {
        "role": "user",
        "content": "Only Fill the [MASK] and directly return the filled sequence: [MASK] performances of [MASK] children , untrained in [MASK] , have an honesty and dignity [MASK] breaks your heart ."
    }
]

# Function to get response for each message
def get_response(message):
    chat_completion = client.chat.completions.create(
        messages=[message],
        model="gpt-4o",
    )
    return chat_completion.choices[0].message.content

# Get responses for each message
responses = [get_response(message) for message in messages]

# Print each response
for response in responses:
    print(response)

It is just too bad this film's story does not live up to its potential.
like changing world of cinema film, Hartley created a masterpiece but didn't know how to handle it.
while "Charade" has definite drawbacks -- like a rather unbelievable love story and a meandering ending -- this classic 60s caper film is a riveting, brisk entertainment.
The performances of young children, untrained in acting, have an honesty and dignity that breaks your heart.


In [58]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me a joke."},
    {"role": "assistant", "content": "Why did the scarecrow win an award? Because he was outstanding in his field!"},
    {"role": "user", "content": "Tell me another one."}
]

# 调用 completions.create 方法
response = client.chat.completions.create(
    model="gpt-4",
    messages=messages
)

# 打印响应
print(response)

ChatCompletion(id='chatcmpl-9ekneC3pJEat9L5mcBasY6vw3Enl3', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure, here's another one for you. Why don't scientists trust atoms? Because they make up everything!", role='assistant', function_call=None, tool_calls=None))], created=1719500046, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=22, prompt_tokens=52, total_tokens=74))


In [49]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."},
    {"role": "user", "content": "Only Fill the [MASK]: [MASK] [MASK] [MASK] [MASK] movie hath twists worthy of David Mamet, and is exceeding mighty for its clever plot."}
  ]
)

In [56]:
print(completion)

ChatCompletion(id='chatcmpl-9ekih4YTinmCDJ1quykVkYejfHA66', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='In the realm of code where loops entwine,\nThere dwells a method both subtle and divine,\nRecursion, a technique so elegantly profound,\nIn programming lands, its praises resound.\n\nLike a mirror reflecting its own reflection,\nIt calls itself, birthing a recursive connection,\nA function that within itself sneaks a peek,\nUnfolding mysteries layer by layer, so sleek.\n\nJust as a tale with twists galore,\nRecursion recurs, opening new doors,\nEach iteration a scene, a frame in the show,\nUnfolding the story with each recursive throw.\n\nA loop that loops within its own embrace,\nUnraveling logic with finesse and grace,\nIn the dance of data and endless repeat,\nRecursion whispers, “The circle is sweet.”\n\nSo embrace this concept, both clever and grand,\nIn the symphony of code, let recursion stand,\nA poetic flow of logic, a

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

pretrained_dir = 'models/huggingface/internlm/internlm2-chat-1_8b-sft'
tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, trust_remote_code=True)
# Set `torch_dtype=torch.float16` to load model in float16, otherwise it will be loaded as float32 and cause OOM Error.
model = AutoModelForCausalLM.from_pretrained(pretrained_dir, torch_dtype=torch.float16, trust_remote_code=True).cuda()
model = model.eval()
response, history = model.chat(tokenizer, "hello", history=[])
print(response)
# Hello! How can I help you today?
response, history = model.chat(tokenizer, "please provide three suggestions about time management", history=history)
print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

你好，很高兴为您服务。请问有什么我可以帮助您的吗？
当涉及到时间管理时，以下是一些建议：

1. 制定一个计划：在一天开始之前，制定一个计划，列出您需要完成的任务。这可以帮助您更好地组织时间并确保您在规定时间内完成任务。

2. 优先考虑重要的任务：将任务按重要性和紧急性排序，然后优先处理最重要的任务。这可以帮助您避免浪费时间在次要任务上。

3. 避免分散注意力：在处理任务时，避免分散注意力。关闭社交媒体和电子邮件通知，并尝试在处理任务时集中注意力。这可以帮助您更快地完成任务并提高效率。


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

pretrained_dir = '/home/fei/NLPLab/Phi-3-mini-4k-instruct'
model = AutoModelForCausalLM.from_pretrained(
    pretrained_dir, 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained(pretrained_dir)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
messages = [
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.1,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

The model 'Phi3ForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyForCaus

 To solve the equation 2x + 3 = 7, follow these steps:

1. Subtract 3 from both sides of the equation: 2x + 3 - 3 = 7 - 3
2. Simplify: 2x = 4
3. Divide both sides by 2: 2x/2 = 4/2
4. Simplify: x = 2

So, the solution to the equation 2x + 3 = 7 is x = 2.


In [14]:
prompt = 'Replace [MASK] with possible word to make the text fluent and natural. The text is: '
ori_text = 'This cunning caper tq movie hath twists worthy of David Mamet, and is exceeding mighty for the thinking audience.'
text = 'Replace [MASK] with possible word to make the text fluent and natural. The text is: {{This cunning caper tq movie hath twists worthy of David Mamet, and is exceeding mighty for [MASK] [MASK] [MASK].}}. Please return the replaced text with the format: Replaced Text: {{}}'
# response, history = model.chat(tokenizer, prompt + text, history=history)
output = pipe([{"role": "user", "content": text}], **generation_args)
print(output[0]['generated_text'])

/home/fei/NLPLab/miniconda3/envs/textGuard39/lib/python3.9/site-packages/transformers-4.40.2-py3.9.egg/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 Replaced Text: {{This cunning caper movie hath twists worthy of David Mamet, and is exceeding mighty for its intricate plot, compelling characters, and masterful direction.}}


In [3]:
response

'The movie has twists worthy of David Mamet, and is exceeding mighty for the thinking audience.'

In [37]:
from transformers import AutoTokenizer, BertForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

inputs = tokenizer("Downloads always resume [MASK] [MASK]. If you want to force a new download, use `force_download=True`.", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)

print(predicted_token_id)
print(tokenizer.convert_ids_to_tokens(predicted_token_id))
# labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]
# # mask labels of non-[MASK] tokens

# labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

# outputs = model(**inputs, labels=labels)

/home/fei/NLPLab/miniconda3/envs/textGuard39/lib/python3.9/site-packages/huggingface_hub-0.23.0-py3.9.egg/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a B

tensor([2044, 2101])
['after', 'later']
